## **Estructura de sub-etapas del procesamiento del habla:**

---

```
├── Input Layer
│   ├── Audio Capture
│   └── ASR (Speech-to-Text)
├── NLP Layer
│   ├── Text Preprocessing
│   ├── Intent Detection
│   ├── Entity Recognition
│   └── Sentiment Analysis
├── Dialogue Management
│   ├── State Management
│   ├── Response Generation
│   ├── Response Selection
│   └── Context Handling
├── Output Layer
│   ├── TTS (Text-to-Speech)
│   └── Audio Playback

```


### **Estructura operativa para cada sub-etapa:**

**Selección de técnicas de modelado:** En este caso se utilizara [] para la etapa de reconocimiento de voz, [] para el procesamiento y analisis de texto y [] para la conversion de texto a voz.

**Generación de un diseño de comprobación:** Para elegir el modelo correcto, este devera ser el que pondere mas alto en un promedio de las diferentes metricas de evaluacion del modelo.

**Generación de los modelos:** Se definiran y configuraran los parametros del modelo para pasar a la etapa de ejecucion y descripcion.

**Evaluación del modelo:** Se evaluaran los diferentes modelos de manera individual en busqueda de optimizar sus parametros y escoger la combinacion mas optima entre modelos / parametros individuales.

In [ ]:
# Execute if use colab and you need import files
from google.colab import files

In [ ]:
# Execute if use colab and you need import files
drive.mount('/content/drive')

In [ ]:
# Execute if use colab
%%bash
apt-get update
apt-get install python3
apt-get install -venv
python3 -m venv proc_habla
source proc_habla/bin/activate

In [1]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

## **1. Entrada de Usuario (Reconocimiento del Habla)**

In [121]:
!pip3 install SpeechRecognition
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub
!pip install pocketsphinx

In [134]:
import speech_recognition as sr
import numpy as np
import wave
import pandas as pd
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment
import subprocess
import zipfile
import os

In [139]:
# Ruta del archivo zip descargado
ruta_modelo_comprimido_pocketsphinx_esp = "../datos/brutos/modelos_proc_habla/modelo_esp.zip"

# Directorio donde se colocarán los archivos descomprimidos
ruta_modelo_descomprimido_pocketsphinx_esp = "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/"

In [143]:
# Descomprimir el archivo zip
with zipfile.ZipFile(ruta_modelo_comprimido_pocketsphinx_esp, 'r') as zip_ref:
    zip_ref.extractall(ruta_modelo_descomprimido_pocketsphinx_esp)

print("Archivos descomprimidos en:", ruta_modelo_descomprimido_pocketsphinx_esp)

Archivos descomprimidos en: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/


In [132]:
try:
    subprocess.run(["sudo", "apt-get", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en sistemas basados en Debian.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)
try:
    subprocess.run(["brew", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en macOS utilizando Homebrew.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


FFmpeg instalado exitosamente en sistemas basados en Debian.
FFmpeg instalado exitosamente en macOS utilizando Homebrew.


To reinstall 7.0.1, run:
  brew reinstall ffmpeg


## **Módulo de Captura de Audio:**

*Bugs*

- Recibo algun id de consulta, cliente? (lo necesito para gestionar la consulta de manera eficiente)
- Definir como se llamara a la funcion chatbot
    - Definir el anidamiento de funciones, para que con solo llamar a chatbot, me entregue el resultado (lista de productos limpia)

In [100]:
SAMPLE_RATE = 44100  # Tasa de muestreo
DURATION = 10  # Duración de la grabación en segundos
AUDIO_FILES_PATH = "../datos/brutos/audios_proc_habla/"
FILENAME = "output"  # Nombre del archivo de salida
EXTENCION_ENTRADA = '.wav'
EXTENCION_SALIDA = '.aiff'

In [101]:
ruta_audio_entrada = AUDIO_FILES_PATH + FILENAME + EXTENCION_ENTRADA
ruta_audio_entrada_convertido = AUDIO_FILES_PATH + 'waw_conv_' + FILENAME + EXTENCION_SALIDA

In [63]:
columns_consultas_df = ['id_consulta', 'id_cliente', 'formato_consulta', 'transcripción_audio', 'entrada_texto', 'productos_detectados']

In [64]:
consultas_df = pd.DataFrame(columns=columns_consultas_df)

In [102]:
def record_audio(audio_file_path = AUDIO_FILES_PATH, filename = FILENAME, duration = DURATION, sample_rate = SAMPLE_RATE):
    print("Grabando...")
    # Grabar audio con 1 canal (mono)
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Esperar a que termine la grabación
    print("Grabación finalizada")
    write(ruta_audio_entrada, sample_rate, recording)
    return filename

In [103]:
def convert_to_wav(input_file = ruta_audio_entrada, output_file = ruta_audio_entrada_convertido):
    sound = AudioSegment.from_file(input_file)
    sound.export(output_file, format="aiff")

In [77]:
def chatbot():
    record_audio()

In [123]:
chatbot()

Grabando...
Grabación finalizada


In [124]:
# Uso de la función convert_to_wav
convert_to_wav(ruta_audio_entrada, ruta_audio_entrada_convertido)

## **Reconocimiento del Habla (ASR - Automatic Speech Recognition):**

In [125]:
# Crear un objeto Recognizer
recognizer = sr.Recognizer()

In [144]:
def recognize_speech():
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(ruta_audio_entrada_convertido) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_sphinx(audio_data, language="es-ES")
            print("Texto reconocido:", text)
    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print("Error al solicitar resultados de reconocimiento de voz; {0}".format(e))

In [145]:
# Ejecutar la función para reconocer el discurso
recognize_speech()

Error al solicitar resultados de reconocimiento de voz; missing PocketSphinx language model parameters directory: "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/acoustic-model"


In [107]:
# Abrir el archivo de audio
def transcribe_audio(audio_lang = 'es-ES'):
    
    with sr.AudioFile(ruta_audio_entrada_convertido) as source:
        # Escuchar el audio (en inglés)
        audio = recognizer.record(source)

        # Utilizar Google Speech Recognition para transcribir el audio
        try:
            text = recognizer.recognize_google(audio, audio_lang)
            print("Transcripción: ", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print("Error al solicitar resultados del servicio Google Speech Recognition; {0}".format(e))

In [108]:
transcribe_audio()

Error al solicitar resultados del servicio Google Speech Recognition; recognition request failed: Forbidden


In [ ]:
# Transcribir y mostrar el resultado
text = transcribe_audio(audio_file_path, audio_lang)

## **2. Procesamiento del Lenguaje Natural (NLP)**

### **Preprocesamiento de Texto:**



**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

**Tokenización**

**Eliminación de stop words.**


**Lematización y stemming**

### **Análisis de Texto:**


Análisis de Sentimiento: Determinar la emoción o tono del texto.


Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


Extracción de Entidades (Entity Recognition): Extraer información relevante del texto, como nombres, fechas, ubicaciones, etc.

## **3. Gestión del Diálogo**

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

## 4. Salida de Usuario (Text-to-Speech)


### Conversión de Texto a Voz (TTS): Utilizar servicios como Google Text-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


### Reproducción de Audio: Entregar la respuesta de voz al usuario.